In [4]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set_theme(style="darkgrid")
warnings.filterwarnings("ignore")


## RNN

In [2]:
x: x_embd, 1
wah: h_dim, x_embd
(wah dot x)

whh: hdn, hdm 
why: y_dim, hdm 


SyntaxError: invalid syntax (2493403685.py, line 1)

In [34]:
class RNN:
    def __init__(self, x_embd, h_embd, y_embd, lr=1e-3):
        self.x_embd = x_embd
        self.h_embd = h_embd
        self.y_embd = y_embd
        self._initialize_parameters(x_embd, h_embd, y_embd)
        self.lr = lr

    def _initialize_parameters(self, 
                               x_embd,
                               h_embd,
                               y_embd):
        """
        Initializing parameters for our RNN
        """
        Wxh = np.random.randn(h_embd, x_embd) * 0.01
        Whh = np.random.randn(h_embd, h_embd) * 0.01
        Why = np.random.randn(y_embd, h_embd) * 0.01
        bh = np.random.randn(h_embd, 1)
        by = np.random.randn(y_embd, 1)

        self.parameters = {
            "Wxh": Wxh,
            "Whh": Whh,
            "Why": Why,
            "bh": bh,
            "by": by,
        }

        self.gradients = dict()

        self.gradients["Wxh"] = np.zeros_like(self.parameters["Wxh"])
        self.gradients["Why"] = np.zeros_like(self.parameters["Why"])
        self.gradients["Whh"] = np.zeros_like(self.parameters["Whh"])
        self.gradients["by"] = np.zeros_like(self.parameters["by"])
        self.gradients["bh"] = np.zeros_like(self.parameters["bh"])
        self.gradients["dnext_h"] = np.zeros_like(self.parameters["bh"])


    def _softmax(self, x):
        """
        x: numpy array of shape (-1, 1)
        """
        temp = np.exp(x - np.max(x))
        return temp/ temp.sum(axis=0)

    def _mse(self, x):
        pass
    
    def __one_step_forward(self, h_prev, x):
        """
        Does one step forward for the RNN
        Returns: logits
        """
        h = np.tanh(np.dot(self.parameters["Wxh"], x) + np.dot(self.parameters["Whh"], h_prev) + self.parameters["bh"]) # (h_embd, 1)
        logits = np.dot(self.parameters["Why"], h) + self.parameters["by"] # (y_embd, 1)
        # preds = self._softmax(logits)

        return h, logits
    
    def forward(self, X, Y):
        """
        Implementation of a full forward pass of the RNN
        Args:
            X: list of indices (C, 1), Here C is the nubmer of examples
            Y: list of indices (C, 1)

        """
        self.x, self.y_hat, self.h, self.y = {}, {}, {}, {}

        # zeroth hidden state (h0)
        self.h[-1] = np.zeros((self.h_embd, 1))

        for i in range(len(X)):
            self.x[i] = np.zeros((self.x_embd, 1))

            # Make it a one-hot vector now
            self.x[i][X[i]] = 1


            # ----------------
            # same for y
            self.y[i] = np.zeros((self.y_embd, 1))
            self.y[i][Y[i]] = 1
            # -----------------


            self.h[i], self.y_hat[i] = self.__one_step_forward(h_prev=self.h[i-1],
                                                               x=self.x[i])
            
            # loss +=  Optional...

    def _one_step_backward(self, x, h, h_prev, dy):
        """
        Args:
            dy: dL/d y_pred
        One backward step, and accumulation of the needed gradients
        equations are:
        
        _h(t) = Wxh @ x + Whh @ h_prev + bh
        h(t) = tanh(_h(t)) + dnext_h

        # --------------------
        ### IMPORTANT HERE
        The dnext_h is the contribution of the next hidden state i.e., h(t+1)
        Now the equations...
        h(t+1) = Whh X h(t)
        now we already know dL/ dh(t+1) which is simply dnext_h
        so dh(t+1)/ dh(t) = Whh
        now dL/ dh(t) = Whh X dnext_h (which is form the succeeding hidden layers.)
        that's why we have to add it like this :).
        # --------------------

        y_pred(t) = Why @ h(t) + by

        Loss = (y - y_pred) ** 2/2 (For simplicity rn)

        Now for gradients:
        
        dL/d y_pred = (y - y_pred) -> (y_embd, 1)
        dL/d Why = (y - y_pred) @ h(t).T -> (y_embd, 1) X (1, h_embd) -> (y_embd, h_embd)
        dL/d by = (y - y_pred)

        dL/d h(t) = Why.T @ (y - y_pred) -> (h_embd, y_embd) X (y_embd, 1) -> (h_embd, 1)
        dL/d _h(t) = dL/ d h(t) * (1 - h(t) ** 2) -> (h_embd, 1)
        dL/d Wxh = dL/d _h(t) @ x.T -> (h_embd, 1) @ (1, x_embd) -> (h_embd, x_embd)
        dL/d bh = dL/d _h(t) -> (h_embd, 1)

        dL/d Whh = dL/d _h(t) @ h_prev.T -> (h_embd, 1)

        """
        self.gradients["Why"] += dy @ h.T
        self.gradients["by"] += dy

        dL_d_h = self.parameters["Why"].T @ dy + self.parameters["Whh"] @ self.gradients["dnext_h"]
        dL_d_h_ = dL_d_h * (1 - h**2) # _h(t)
        self.gradients["Wxh"] += dL_d_h_ @ x.T
        self.gradients["Whh"] += dL_d_h_ @ h_prev.T
        self.gradients["bh"] += dL_d_h_
        self.gradients["dnext_h"] = dL_d_h_

    def rnn_backward(self, X, Y):
        """
        Performs full BPT (Backpropagation through time)
        Args:
            X: list of indices (C, 1), Here C is the nubmer of examples
            Y: list of indices (C, 1)
        """
        for t in reversed(range(len(X))):
            dy = self.y[t] - self.y_hat[t]
            #  dy = dL/ dy_pred = (y - y_pred)

            self._one_step_backward(x=self.x[t],
                                    h=self.h[t],
                                    h_prev=self.h[t-1],
                                    dy=dy)
            
    def update_parameters(self, lr=None):
        """
        Does simple SGD optimization
        """
        if lr == None:
            lr = self.lr
        self.parameters["Wxh"] += -lr * self.gradients["Wxh"]
        self.parameters["Whh"] += -lr * self.gradients["Whh"]
        self.parameters["Why"] += -lr * self.gradients["Why"]
        self.parameters["by"] += -lr * self.gradients["by"]
        self.parameters["bh"] += -lr * self.gradients["bh"]


In [35]:
m = RNN(x_embd=6,
        h_embd=7,
        y_embd=5)

In [36]:
X = [1, 3, 3]
Y = [0, 2, 1]
m.forward(X, Y)

In [37]:
m.rnn_backward(X, Y)

In [38]:
m.gradients["Whh"]

array([[-1.30189586e-03,  3.93118968e-02,  3.03283071e-02,
        -7.44958060e-03,  3.93762113e-02, -8.64504447e-03,
        -4.28397073e-02],
       [-3.89463541e-04,  1.14861272e-02,  8.91501297e-03,
        -2.17697413e-03,  1.15003535e-02, -2.49277095e-03,
        -1.24978334e-02],
       [-4.90307109e-04,  1.54427692e-02,  1.17888486e-02,
        -2.92555801e-03,  1.54786526e-02, -3.47307680e-03,
        -1.68728942e-02],
       [ 2.23205830e-05, -3.24182703e-03, -1.99781533e-03,
         6.10951372e-04, -3.28990287e-03,  1.02334208e-03,
         3.71120698e-03],
       [-9.76687698e-04,  2.88635707e-02,  2.23907771e-02,
        -5.47045453e-03,  2.89003249e-02, -6.27139551e-03,
        -3.14100873e-02],
       [ 3.06151748e-04, -1.15918939e-02, -8.48291946e-03,
         2.19357357e-03, -1.16499583e-02,  2.83294499e-03,
         1.27952785e-02],
       [-2.15324198e-04,  6.46085453e-03,  4.99245836e-03,
        -1.22438194e-03,  6.47074077e-03, -1.41583812e-03,
        -7.0377922